# Extracting Question and Answer pairs from raw plain text using GPT

## Load Data

In [1]:
!pip install -qU datasets

In [2]:
from datasets import load_dataset

In [ ]:
data = load_dataset("MuhammadHelmy/nafsy", split="train")

## Chat with GPT-3.5-Turbo

In [ ]:
!pip install -qU openai

In [15]:
import openai
from google.colab import userdata

In [16]:
openai.api_key = userdata.get('OPENAI_API_KEY')

**GPT Prompt:**

{contex}

Provide 2 questions and answers based on the text above. The questions must begin with "في سياق ...". The answers should borrow, verbatim, from the text above. Vary the style and format of questions. Respond in plain text on a new line for each question and answer. Leave out the question numbers and bullets. Here is a sample question and answer:

في سياق علم النفس وتربية الأطفال، ما هو أسلوب الدقيقة الواحدة في تربية الأطفال؟\nأسلوب الدقيقة الواحدة في تربية الأطفال هو أسلوب حديث يهدف إلى التعامل مع سلوك الأطفال السلبي بطريقة فعالة ومحبة في دقيقة واحدة فقط. يعتمد هذا الأسلوب على جعل الأطفال يشعرون بعدم الرضا عن تصرفهم الخاطئ، ولكن بالرضا عن أنفسهم. يتم ذلك من خلال التعبير بصدق عن مشاعر الغضب والحزن تجاه سلوك الطفل، ثم تأكيد حبك وتقديم الدعم بعد ذلك. يتم تطبيق هذا الأسلوب من خلال تحديد السلوك السلبي والتعبير عن الغضب والحزن بوضوح في النصف الأول من الدقيقة، ثم التركيز على الهدوء وتقديم الدعم والمحبة في النصف الثاني من الدقيقة.

#### Create Train Data

In [6]:
import pandas as pd
from datasets import Dataset

In [ ]:
def create_train_prompt(context):
    prompt = f"""
    {context}

    Provide 2 questions and answers based on the text above. The questions must begin with "في سياق ...". The answers should borrow, verbatim, from the text above. Vary the style and format of questions. Respond in plain text on a new line for each question and answer. Leave out the question numbers and bullets. Here is a sample question and answer:

    في سياق علم النفس وتربية الأطفال، ما هو أسلوب الدقيقة الواحدة في تربية الأطفال؟\nأسلوب الدقيقة الواحدة في تربية الأطفال هو أسلوب حديث يهدف إلى التعامل مع سلوك الأطفال السلبي بطريقة فعالة ومحبة في دقيقة واحدة فقط. يعتمد هذا الأسلوب على جعل الأطفال يشعرون بعدم الرضا عن تصرفهم الخاطئ، ولكن بالرضا عن أنفسهم. يتم ذلك من خلال التعبير بصدق عن مشاعر الغضب والحزن تجاه سلوك الطفل، ثم تأكيد حبك وتقديم الدعم بعد ذلك. يتم تطبيق هذا الأسلوب من خلال تحديد السلوك السلبي والتعبير عن الغضب والحزن بوضوح في النصف الأول من الدقيقة، ثم التركيز على الهدوء وتقديم الدعم والمحبة في النصف الثاني من الدقيقة."""

    return prompt

In [ ]:
# # Ask GPT to form question and answer pairs from the raw data
with open("nafsy_QA_train.txt", mode='a', encoding='utf-8') as file:
    for ex in data:
        try:
            prompt = create_train_prompt(ex['content'])

            response = openai.chat.completions.create(
                model = "gpt-3.5-turbo",
                messages = [
                    {"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user", "content":prompt}
                ],
                temperature = 0.5
            )
            response_text = response.choices[0].message.content.strip()

            # print(f"iteration {i}:\n {response_text}")
            file.write(f"{response_text}\n")

        except Exception as e:
            print(f"An error occured: {e}")

In [ ]:
QA_train = pd.read_csv('nafsy_QA_train.txt', header=None)

In [ ]:
# convert to (prompt | response) format

QA_train_df = pd.DataFrame(columns=['prompt', 'response'])

for i in range(1, len(QA_train), 2):
    QA_train_df.loc[i//2] = [QA_train[0][i-1], QA_train[0][i]]

QA_train_df.head()

In [ ]:
# Drop noisy rows
QA_train_df = QA_train_df.drop(145)

In [ ]:
# Switching mixed values between prompt and response
QA_train_df.loc[145:len(QA_train_df), ['prompt','response']] = QA_train_df.loc[145:len(QA_train_df), ['response','prompt']].values

In [ ]:
# Convert to csv
QA_train_df.to_csv('nafsy_QA_train.csv', index=False)

In [ ]:
# Drop rows to modify synchronization between prompt and response
QA_train_df['prompt'] = QA_train_df['prompt'].drop(233)
QA_train_df['response'] = QA_train_df['response'].drop(146)

In [ ]:
# Create prompt and response individually then concatenate them to adjust indexing

prompt = QA_train_df[['prompt']].dropna().reset_index(drop=True)
response = QA_train_df[['response']].dropna().reset_index(drop=True)

QA_train_df = pd.concat([prompt, response], axis=1)

In [ ]:
# Shuffle Dataframe
QA_train_df = QA_train_df.sample(frac=1.0).reset_index(drop=True)
QA_train_df

In [ ]:
# Save as csv
QA_train_df.to_csv("nafsy_QA_train.csv", index=False)

In [ ]:
# Push to Hugging Face
QA_train_df = pd.read_csv("nafsy_QA_train.csv")
QA_train_hf = Dataset.from_pandas(QA_train_df)
QA_train_hf.push_to_hub("MuhammadHelmy/nafsy-QA", split="train")

#### Create Test Data

In [ ]:
def create_test_prompt(context):
    prompt = f"""
    {context}

    Provide 1 (question and answer) based on the text above. The questions must begin with "في سياق ...". The answers should NOT borrow verbatim from the text above. Vary the style and format of questions. Respond in plain text on a new line for each question and answer. Leave out the question numbers and bullets. Here is a sample question and answer:

    في سياق علم النفس وتربية الأطفال، ما هو أسلوب الدقيقة الواحدة في تربية الأطفال؟\nأسلوب الدقيقة الواحدة في تربية الأطفال هو أسلوب حديث يهدف إلى التعامل مع سلوك الأطفال السلبي بطريقة فعالة ومحبة في دقيقة واحدة فقط. يعتمد هذا الأسلوب على جعل الأطفال يشعرون بعدم الرضا عن تصرفهم الخاطئ، ولكن بالرضا عن أنفسهم. يتم ذلك من خلال التعبير بصدق عن مشاعر الغضب والحزن تجاه سلوك الطفل، ثم تأكيد حبك وتقديم الدعم بعد ذلك. يتم تطبيق هذا الأسلوب من خلال تحديد السلوك السلبي والتعبير عن الغضب والحزن بوضوح في النصف الأول من الدقيقة، ثم التركيز على الهدوء وتقديم الدعم والمحبة في النصف الثاني من الدقيقة."""

    return prompt

In [ ]:
# # # Ask GPT to form question and answer pairs from the raw data
with open("nafsy_QA_test.txt", mode='a', encoding='utf-8') as file:
    for ex in data:
        try:
            prompt = create_test_prompt(ex['content'])

            response = openai.chat.completions.create(
                model = "gpt-3.5-turbo",
                messages = [
                    {"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user", "content":prompt}
                ],
                temperature = 0.5
            )
            response_text = response.choices[0].message.content.strip()

            # print(f"iteration {i}:\n {response_text}")
            file.write(f"{response_text}\n")

        except Exception as e:
            print(f"An error occured: {e}")

In [12]:
QA_test = pd.read_csv('nafsy_QA_test.txt', header=None)

In [ ]:
# convert to (prompt | response) format

QA_test_df = pd.DataFrame(columns=['prompt', 'response'])

for i in range(1, len(QA_test), 2):
    QA_test_df.loc[i//2] = [QA_test[0][i-1], QA_test[0][i]]

QA_test_df

In [ ]:
# Shuffle Dataframe
QA_test_df = QA_test_df.sample(frac=1.0).reset_index(drop=True)
QA_test_df

In [ ]:
# Convert to csv
QA_test_df.to_csv('nafsy_QA_test.csv', index=False)

In [ ]:
# Push to Hugging Face
QA_test_df = pd.read_csv("nafsy_QA_test.csv")
QA_test_hf = Dataset.from_pandas(QA_test_df)
QA_test_hf.push_to_hub("MuhammadHelmy/nafsy-QA", split="test", )